<a href="https://colab.research.google.com/github/Buzon-coder/QRT-Electricity-price/blob/main/QRT_Electricity_Price2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
#from google.colab import files
#uploaded = files.upload()
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import spearmanr, pearsonr

In [25]:
X_test = pd.read_csv("/content/drive/MyDrive/Colab data/X_test_final.csv")
X_train = pd.read_csv("/content/drive/MyDrive/Colab data/X_train_NHkHMNU.csv")
y_train = pd.read_csv("/content/drive/MyDrive/Colab data/y_train_ZAN5mwg.csv")

In [26]:
common_id = set(X_train["ID"]) & set(X_test["ID"])
print("Nombre d'ID' en commun entre TRAIN et TEST:",len(common_id))

common_days = set(X_train["DAY_ID"]) & set(X_test["DAY_ID"])
print("Nombre de jours en commun entre TRAIN et TEST:",len(common_days))

Nombre d'ID' en commun entre TRAIN et TEST: 0
Nombre de jours en commun entre TRAIN et TEST: 0


In [27]:
# je trie X_train par DAY_ID croissant (donc par dates) et j'aligne les ID de y_train avec ceux de DAY_ID

X_train = X_train.sort_values(["DAY_ID", "COUNTRY"]).reset_index(drop=True)
y_train = y_train.set_index("ID").loc[X_train["ID"]].reset_index()
assert all(X_train["ID"].values == y_train["ID"].values)
print("Les ID sont bien alignés avec X_train")

Les ID sont bien alignés avec X_train


In [28]:
#print(X_train.head())

J'encode les features de X_train qui ne sont pas des int ou des float (en l'occurence, seul "COUNTRY" n'est pas un float ou un int, donc c'est la seule colonne que j'encode)

In [29]:
##j'encode la colonne COUNTRY en 2 colonnes : COUNTRY_FR et COUNTRY_DE, et je supprime la colonne COUNTRY
X_train_encode = X_train.copy()
X_train_encode["COUNTRY_FR"] = (X_train_encode["COUNTRY"] == "FR").astype(int)
X_train_encode["COUNTRY_DE"] = (X_train_encode["COUNTRY"] == "DE").astype(int)
X_train_encode = X_train_encode.drop(columns = ["COUNTRY"])
#print(X_train_encode.head())

On regarde maintenant les corrélations (en valeur absolue) entre les features et les TARGET

In [30]:
corrs = X_train_encode.corrwith(y_train["TARGET"], method = 'spearman') # Remarque : la fonction corrwith calcule la corrélation entre les features de X_train et les valeurs TARGET de y_train ligne par ligne
                                                                        # Remarque 2 : on utilise la corrélation de Spearman car c'est la corrélation de l'énoncé
corrs_abs = corrs.abs().sort_values(ascending=False) # on regarde les valeurs absolues des corrélations des features avec la TARGET. En effet, une corrélation négative mais avec une valeur absolue élevée est aussi une corrélation forte, c'est simplement
                                                     # qu'elle agit dans le sens inverse de l'évolution du prix du futures (qd feature augmente, cible diminue, ou l'inverse). exemple : si augmentation du vent, prix diminue car on a + d'électricité
#print(corrs_abs.head(20))
#print(corrs_abs.tail(18))

J'obtiens que les features les plus corrélées (en valeur absolue) de X_train et avec les TARGET (de y_train) sont par ordre décroissant : DE_NET_IMPORT, DE_NET_EXPORT, DE_WINDPOW, DE_RESIDUAL_LOAD, FR_WINDPOW,DE_HYDRO, DE_GAS, CARBON_RET, DE_WIND, DE_COAL, GAS_RET, FR_HYDRO,FR_WIND, DE_CONSUMPTION, FR_RAIN, FR_DE_EXCHANGE, DE_FR_EXCHANGE, FR_COAL, FR_TEMP, DE_LIGNITE.


De même, j'obtiens les features les moins corrélées (en valeur absolue) de X_train avec les TARGET sont (de la plus corrélées à la moins corrélée) : DE_RAIN, DE_SOLAR, FR_SOLAR, FR_CONSUMPTION, ID, COUNTRY_FR, COUNTRY_DE, DE_TEMP, FR_RESIDUAL_LOAD, COAL_RET, FR_GAS, FR_NET_EXPORT, FR_NET_IMPORT, DE_NUCLEAR, DAY_ID, FR_NUCLEAR.

Regarder les corrélations permet de comprendre quelles features influencent TARGET, et permet d'interpréter phyisquement le modèle. Ces features représentent les effets dominants.

Nettoyage des données

In [31]:
# On regarde si les features de X_train_encode contiennent des NaN dans le pays qui leur est associé.
# Tout d'abord, on regarde quelles features contiennent des NaN

print(X_train_encode.isna().sum())

ID                    0
DAY_ID                0
DE_CONSUMPTION        0
FR_CONSUMPTION        0
DE_FR_EXCHANGE       25
FR_DE_EXCHANGE       25
DE_NET_EXPORT       124
FR_NET_EXPORT        70
DE_NET_IMPORT       124
FR_NET_IMPORT        70
DE_GAS                0
FR_GAS                0
DE_COAL               0
FR_COAL               0
DE_HYDRO              0
FR_HYDRO              0
DE_NUCLEAR            0
FR_NUCLEAR            0
DE_SOLAR              0
FR_SOLAR              0
DE_WINDPOW            0
FR_WINDPOW            0
DE_LIGNITE            0
DE_RESIDUAL_LOAD      0
FR_RESIDUAL_LOAD      0
DE_RAIN              94
FR_RAIN              94
DE_WIND              94
FR_WIND              94
DE_TEMP              94
FR_TEMP              94
GAS_RET               0
COAL_RET              0
CARBON_RET            0
COUNTRY_FR            0
COUNTRY_DE            0
dtype: int64


je veux voir ce que j'ai à une ligne telle que COUNTRY == "DE" et dans la colonne FR_GAS.

In [32]:
#print(X_train_encode.head())
#print(X_train[])
day_id_test = X_train["DAY_ID"].value_counts().index[0]
print(X_train["DAY_ID"].value_counts())
# on voit que le jour 1213 (DAY_ID = 1213) apparaît 2 fois dans le dataset
exemple = X_train[X_train["DAY_ID"] == 1213][["DAY_ID", "FR_GAS", "DE_GAS"]]
print(exemple)

DAY_ID
1213    2
1184    2
1183    2
1182    2
1181    2
       ..
1176    1
1177    1
1186    1
0       1
1215    1
Name: count, Length: 851, dtype: int64
      DAY_ID    FR_GAS    DE_GAS
1491    1213  1.549682  1.781433
1492    1213  1.549682  1.781433


--> on voit que la colonne FR_GAS contient 2 fois les mêmes valeurs pour chaque DAY_ID